# Tech Intro

## Main tables

In [1]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [2]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = real_estate_list[5] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [3]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

22/11/06 22:01:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/06 22:01:21 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/06 22:01:22 WARN Utils: Service 'SparkUI' could not bind on port 4080. Attempting port 4081.
22/11/06 22:01:22 WARN Utils: Service 'SparkUI' could not bind on port 4081. Attempting port 4082.
22/11/06 22:01:22 WARN Utils: Service 'SparkUI' could not bind on port 4082. Attempting port 4083.
22/11/06 22:01:22 WARN Utils: Service 'SparkUI' could not bind on port 4083. Attempting port 4084.
22/11/06 22:01:22 WARN FairSchedulableBuilder: Fair Sc

## Read the spec-file from Hadoop and show some data sample

In [4]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-----+----------------+----------------+-----------------+----+-----------------------------------------------------------------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+-----------+------------+-----+-----------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------+---------------+------+----------+-----------+
|id   |ag_posting_ad_id|native_realty_id|foreign_realty_id|type|title                                                                        |price    |address                                                                                                                                       |rooms|area |living_area|kitchen_area|floor|total_flo

## Show columns' properties

In [5]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- ag_posting_ad_id: string (nullable = true)
 |-- native_realty_id: string (nullable = true)
 |-- foreign_realty_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: string (nullable = true)
 |-- address: string (nullable = true)
 |-- rooms: string (nullable = true)
 |-- area: string (nullable = true)
 |-- living_area: string (nullable = true)
 |-- kitchen_area: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- total_floor: string (nullable = true)
 |-- build_year: string (nullable = true)
 |-- description: string (nullable = true)
 |-- hide_characteristic: string (nullable = true)
 |-- hide_description: string (nullable = true)
 |-- hide_attachment: string (nullable = true)
 |-- active: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- agent_phone: string (nullable = true)



## Total rows (count)

In [6]:
eda_df.count()

5546

## Simple func to execute and show the result of the SQL-instruction

In [7]:
def execute(select_instruction: str = "select count(*) from realEstate") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from realEstate"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show()

## Create the temp-table-view from the DF (called as "eda_sql")

In [8]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [9]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|    5546|
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [10]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,ag_posting_ad_id,native_realty_id,foreign_realty_id,type,title,price,address,rooms,area,...,floor,total_floor,build_year,description,hide_characteristic,hide_description,hide_attachment,active,agent_name,agent_phone
0,0,0,183,5363,0,4,0,870,612,10,...,18,121,3314,0,0,0,0,0,5251,5249


In [11]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,ag_posting_ad_id,native_realty_id,foreign_realty_id,type,title,price,address,rooms,area,...,floor,total_floor,build_year,description,hide_characteristic,hide_description,hide_attachment,active,agent_name,agent_phone
0,0,0,0,0,0,0,134,0,32,23,...,980,423,0,3,5236,5205,5546,3965,2,0


In [12]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,ag_posting_ad_id,native_realty_id,foreign_realty_id,type,title,price,address,rooms,area,...,floor,total_floor,build_year,description,hide_characteristic,hide_description,hide_attachment,active,agent_name,agent_phone
0,0,0,0,0,0,15,0,22,0,0,...,0,0,9,246,0,0,0,0,3,2


In [13]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
ag_posting_ad_id,0,0,0
native_realty_id,183,0,0
foreign_realty_id,5363,0,0
type,0,0,0
title,4,0,15
price,0,134,0
address,870,0,22
rooms,612,32,0
area,10,23,0


In [14]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [15]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
ag_posting_ad_id,0.000,0.000,0.000
native_realty_id,3.300,0.000,0.000
foreign_realty_id,96.700,0.000,0.000
type,0.000,0.000,0.000
title,0.072,0.000,0.270
price,0.000,2.416,0.000
address,15.687,0.000,0.397
rooms,11.035,0.577,0.000
area,0.180,0.415,0.000


### SQL-instructions

In [51]:
#Find dupes or misunderstanding or useless static values 5 546 total
execute("SELECT id\
        FROM eda_sql\
        WHERE rooms = 9 OR rooms = 0")

execute("SELECT id\
        FROM eda_sql\
        WHERE price = 1000")

execute("SELECT id\
        ,total_floor\
        FROM eda_sql\
        WHERE total_floor > 100")

for col in eda_df.columns: 
    execute("select count(distinct("+ col + ")) from eda_sql")
    
        
execute("SELECT id\
        ,build_year\
        FROM eda_sql\
        WHERE build_year > 2022")

execute("SELECT id\
        ,build_year\
        FROM eda_sql\
        WHERE build_year < 1900")


+-----+
|   id|
+-----+
|11521|
|11999|
| 1510|
| 3427|
| 4562|
|  713|
| 9049|
|10494|
|10855|
|11937|
|12117|
| 1568|
| 2125|
| 2460|
| 3178|
| 6505|
| 6922|
| 7464|
|10201|
|11530|
+-----+
only showing top 20 rows

+-----+
|   id|
+-----+
|11868|
+-----+

+-----+-----------+
|   id|total_floor|
+-----+-----------+
|10245|        181|
| 7237|        120|
| 9160|        106|
+-----+-----------+

+------------------+
|count(DISTINCT id)|
+------------------+
|              5546|
+------------------+

+--------------------------------+
|count(DISTINCT ag_posting_ad_id)|
+--------------------------------+
|                            3071|
+--------------------------------+

+--------------------------------+
|count(DISTINCT native_realty_id)|
+--------------------------------+
|                            5159|
+--------------------------------+

+---------------------------------+
|count(DISTINCT foreign_realty_id)|
+---------------------------------+
|                              182

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [52]:
spark.stop()

I1106 22:26:29.902297 43263 sched.cpp:2174] Asked to stop the driver
I1106 22:26:29.902475 43526 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-53202
